## Persiapan dataset

In [1]:
import os

# Pastikan untuk mengganti path dengan absolute path direktorimu jika baris berikut dijalankan ulang, atau restart kernel.
os.chdir("dataset/txt")
base = "prastyo-sentiment_posneg.txt"

# Buka file input file dan baca baris per baris
input_stream = open(base, "r", encoding="utf8")
input_stream_lines = input_stream.readlines()
input_stream.close()

In [2]:
# Memisahkan teks dan label dari input
text = []
for line in input_stream_lines:
    text.append(line.split("\t")[0])
    
label = []
for line in input_stream_lines:
    label.append(line.split("\t")[1])

In [3]:
# Cek panjang input
print(len(input_stream_lines), len(text))

# Print sampel terakhir
print("\ninput_stream_lines: \n", input_stream_lines[-1:],
      "\n\ntext: \n", text[-1:],
      "\n\nlabel: \n", label[-1:])

1918 1918

input_stream_lines: 
 ['Apapun agama dan kepercayaanmu, sblum tidur, yuk doakan mereka yg dalam prwatan COVID-19, nakes yg menangani, pemerintah dan warga Indonesia agar bersatu visi, bebas dari pandemi ini dalam waktu sesingkat-singkatnya.\tpos\n'] 

text: 
 ['Apapun agama dan kepercayaanmu, sblum tidur, yuk doakan mereka yg dalam prwatan COVID-19, nakes yg menangani, pemerintah dan warga Indonesia agar bersatu visi, bebas dari pandemi ini dalam waktu sesingkat-singkatnya.'] 

label: 
 ['pos\n']


In [4]:
# Buat file output
os.chdir("../../output")
output = os.path.splitext(base)[0]+'-clean.txt'

## Pembersihan data teks dengan **RegEx**

In [5]:
import re

count=0
with open(output, 'w') as f:
    for line in text:
        # Tahap-1: Non-ascii
        res = re.sub(r'[^\x00-\x7F]+',' ', line)
        # Tahap-2: URLs
        res = re.sub(r'http[s]?\:\/\/.[a-zA-Z0-9\.\/\_?=%&#\-\+!]+',' ', res)
        res = re.sub(r'pic.twitter.com?.[a-zA-Z0-9\.\/\_?=%&#\-\+!]+',' ', res)
        # Tahap-3: mentions
        res = re.sub(r'\@([\w]+)',' ', res)
        
        
        # !!! Pilih Salah Satu !!!
        # Tahap-4_alt-1: hapus tagar
        # res = re.sub(r'\#([\w]+)',' ', res)
        # Tahap-4_alt-2: konversi tagar ke kalimat (pemisahan string berdasarkan huruf kapital)**
        res = re.sub(r'((?<=[a-z])[A-Z]|[A-Z](?=[a-z]))', ' \\1', res)
        #res = re.sub(r'([A-Z])(?<=[a-z]\1|[A-Za-z]\1(?=[a-z]))',' \\1', res)
        
        
        # Tahap-5: simbol
        res = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', res)
        # Tahap-6: angka
        res = re.sub(r'[0-9]+','', res)
        # Tahap-7: koreksi duplikasi tiga karakter beruntun atau lebih (contoh. yukkk)
        res = re.sub(r'([a-zA-Z])\1\1','\\1', res)

        # Tahap-8: spasi ganda (atau lebih) menjadi satu spasi
        res = re.sub(' +', ' ', res)
        # Tahap-9: spasi di awal dan akhir kalimat
        res = re.sub(r'^[ ]|[ ]$','', res)
        
        # Tahap-10: konversi ke karakter huruf kecil
        res = res.lower()
        
        # Tulis setiap baris yang sudah dikoreksi ke file output
        # dan mengembalikan label awal
        f.write(str(res+"\t"+label[count]))
        count+=1

## Catatan:
Untuk Tahap-4, pilih salah satu opsi: (1) hapus tagar atau (2) konversi tagar ke teks
<blockquote> **ref: <i>https://stackoverflow.com/questions/1097901/regular-expression-split-string-by-capital-letter-but-ignore-tla</i></blockquote>